In [4]:
import geemap
import ee
import geopandas as gpd
import pandas as pd

In [5]:
ee.Initialize()

In [10]:
#Load the Lake Victoria shapefile Asset
lake = "projects/water-quality-441207/assets/lake_victoria"
lake = ee.FeatureCollection(lake)

In [7]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>

In [11]:
dataset = ee.ImageCollection('MODIS/061/MYD09GA') \
.filter(ee.Filter.date('2018-04-01', '2018-06-01')).mean()
TrueColor143 = \
dataset.select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03'])
TrueColor143Vis = {
    "min": -100.0,
    "max": 8000.0,
}
m.setCenter(-7.03125, 31.0529339857, 2)
m.addLayer(TrueColor143.clip(lake), TrueColor143Vis, 'True Color (143)')

In [24]:
# Add as layer
m.addLayer(lake, {}, 'Lake Victoria')

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2024-11-08')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds).median()
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m.add_layer(dataset.clip(lake), visualization, 'RGB')

In [27]:
# Calculate NDWI
ndwi_bands = ['B3', 'B5']
ndwi = dataset.normalizedDifference(ndwi_bands).rename('ndwi')

# Add NDWI band to the dataset
dataset = dataset.addBands(ndwi)

# Define visualization parameters for NDWI
ndwi_visualization = {
    'min': -1,
    'max': 1,
    'palette': [
        '0000FF',  # Blue
        '00FFFF',  # Cyan
        '00FF00',  # Green
        'FFFF00',  # Yellow
        'FFA500',  # Orange
        'FF0000'   # Red
    ]
}

# Add NDWI layer to the map
m.addLayer(dataset.select('ndwi').clip(lake), ndwi_visualization, 'NDWI')

In [28]:
# Calculate NDCI
band_ndci = ['B4', 'B3']
ndci = dataset.normalizedDifference(band_ndci).rename('ndci')


# Add NDCI and latlon bands to the dataset
dataset = dataset.addBands(ndci)

# Define visualization parameters for NDCI
ndci_visualization = {
    'min': -1,
    'max': 1,
    'palette': [
        '0000FF',  # Blue
        '00FFFF',  # Cyan
        '00FF00',  # Green
        'FFFF00',  # Yellow
        'FFA500',  # Orange
        'FF0000'   # Red
    ]
}

# Add NDCI layer to the map
m.addLayer(dataset.select('ndci').clip(lake), ndci_visualization, 'NDCI')

In [29]:
band_ndti = ['B5', 'B11']
ndti = dataset.normalizedDifference(band_ndti).rename('ndti')

# Add NDVI band to the dataset
dataset = dataset.addBands(ndti)

# Define visualization parameters for NDTI
ndti_visualization = {
    'min': -1,
    'max': 1,
    'palette': [
        '0000FF',  # Blue
        '00FFFF',  # Cyan
        '00FF00',  # Green
        'FFFF00',  # Yellow
        'FFA500',  # Orange
        'FF0000'   # Red
    ]
}

# Add NDVI layer to the map
m.addLayer(dataset.select('ndti').clip(lake), ndti_visualization, 'NDTI')

In [35]:
ndti_dataset = dataset.select('ndti')
out_ndti_stats = "ndti.csv" 

geemap.zonal_stats(
    ndti_dataset,
    lake,
    out_ndti_stats,
    stat_type="MEAN",
    scale= 1000,
    return_fc=False
)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /workspaces/Water-Quality-Monitoring/notebooks/ndti.csv


In [37]:
# Calculate pH
ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(
            dataset.select('B1').divide(dataset.select('B8')))).rename('ph')

# Add pH band to the dataset
dataset = dataset.addBands(ph)

# Define visualization parameters for pH
ph_visualization = {
    'min': 5,
    'max': 9,
    'palette': [
        'FF0000',  # Red
        'FFA500',  # Orange
        'FFFF00',  # Yellow
        '00FF00',  # Green
        '00FFFF',  # Cyan
        '0000FF'   # Blue
    ]
}

# Add pH layer to the map
m.addLayer(dataset.select('ph').clip(lake), ph_visualization, 'pH')

In [ ]:
# Calculate chlorophyll-a (chl_a)
Rrs_red = dataset.select('B3').divide(0.1)
Rrs_blue = dataset.select('B2').divide(0.1)
chl_a = Rrs_red.divide(Rrs_blue).pow(2.72).multiply(10.8).rename('chl_a')

# Add chl_a band to the dataset
dataset = dataset.addBands(chl_a)

# Define visualization parameters for chl_a
chl_a_visualization = {
    'min': 0,
    'max': 50,
    'palette': [
        '0000FF',  # Blue
        '00FFFF',  # Cyan
        '00FF00',  # Green
        'FFFF00',  # Yellow
        'FFA500',  # Orange
        'FF0000'   # Red
    ]
}

# Add chl_a layer to the map
m.addLayer(dataset.select('chl_a').clip(lake), chl_a_visualization, 'Chlorophyll-a')

In [ ]:
# Calculate SSC
ssc = ee.Image(0.0113).multiply(dataset.select('ndwi').pow(3)) \
            .subtract(ee.Image(0.0135).multiply(dataset.select('ndwi').pow(2))) \
            .add(ee.Image(0.0075).multiply(dataset.select('ndwi'))) \
            .add(ee.Image(2.5823)).rename('ssc')

# Add SSC band to the dataset
dataset = dataset.addBands(ssc)

# Define visualization parameters for SSC
vis_params = {'bands': ['ssc'], 'palette': ['#67001f', '#6a011f', '#6d0220', '#700320', '#730421', '#760521', '#790622', '#7c0722', '#7f0823', '#810823', '#840924', '#870a24', '#8a0b25', '#8d0c25', '#900d26', '#930e26', '#960f27', '#991027', '#9c1127', '#9f1228', '#a21328', '#a51429', '#a81529', '#ab162a', '#ae172a', '#b1182b', '#b3192c', '#b41c2d', '#b61f2e', '#b72230', '#b82531', '#ba2832', '#bb2a34', '#bd2d35', '#be3036', '#bf3338', '#c13639', '#c2383a', '#c43b3c', '#c53e3d', '#c6413e', '#c84440', '#c94741', '#cb4942', '#cc4c44', '#ce4f45', '#cf5246', '#d05548', '#d25849', '#d35a4a', '#d55d4c', '#d6604d', '#d7634f', '#d86551', '#da6853', '#db6b55', '#dc6e57', '#dd7059', '#de735c', '#df765e', '#e17860', '#e27b62', '#e37e64', '#e48066', '#e58368', '#e6866a', '#e8896c', '#e98b6e', '#ea8e70', '#eb9172', '#ec9374', '#ee9677', '#ef9979', '#f09c7b', '#f19e7d', '#f2a17f', '#f3a481', '#f4a683', '#f5a886', '#f5aa89', '#f5ac8b', '#f6af8e', '#f6b191', '#f6b394', '#f7b596', '#f7b799', '#f7b99c', '#f8bb9e', '#f8bda1', '#f8bfa4', '#f9c2a7', '#f9c4a9', '#f9c6ac', '#fac8af', '#facab1', '#fbccb4', '#fbceb7', '#fbd0b9', '#fcd3bc', '#fcd5bf', '#fcd7c2', '#fdd9c4', '#fddbc7', '#fddcc9', '#fdddcb', '#fcdecd', '#fcdfcf', '#fce0d0', '#fce2d2', '#fbe3d4', '#fbe4d6', '#fbe5d8', '#fbe6da', '#fae7dc', '#fae8de', '#fae9df', '#faeae1', '#f9ebe3', '#f9ede5', '#f9eee7', '#f9efe9', '#f9f0eb', '#f8f1ed', '#f8f2ef', '#f8f3f0', '#f8f4f2', '#f7f5f4', '#f7f6f6', '#f6f7f7', '#f5f6f7', '#f3f5f6', '#f2f5f6', '#f0f4f6', '#eff3f5', '#edf2f5', '#ecf2f5', '#eaf1f5', '#e9f0f4', '#e7f0f4', '#e6eff4', '#e4eef4', '#e3edf3', '#e1edf3', '#e0ecf3', '#deebf2', '#ddebf2', '#dbeaf2', '#dae9f2', '#d8e9f1', '#d7e8f1', '#d5e7f1', '#d4e6f1', '#d2e6f0', '#d1e5f0', '#cfe4ef', '#cce2ef', '#cae1ee', '#c7e0ed', '#c5dfec', '#c2ddec', '#c0dceb', '#bddbea', '#bbdaea', '#b8d8e9', '#b6d7e8', '#b3d6e8', '#b1d5e7', '#aed3e6', '#acd2e5', '#a9d1e5', '#a7d0e4', '#a5cee3', '#a2cde3', '#a0cce2', '#9dcbe1', '#9bc9e0', '#98c8e0', '#96c7df', '#93c6de', '#90c4dd', '#8dc2dc', '#8ac0db', '#87beda', '#84bcd9', '#81bad8', '#7eb8d7', '#7bb6d6', '#78b4d5', '#75b2d4', '#71b0d3', '#6eaed2', '#6bacd1', '#68abd0', '#65a9cf', '#62a7ce', '#5fa5cd', '#5ca3cb', '#59a1ca', '#569fc9', '#529dc8', '#4f9bc7', '#4c99c6', '#4997c5', '#4695c4', '#4393c3', '#4291c2', '#408fc1', '#3f8ec0', '#3e8cbf', '#3c8abe', '#3b88be', '#3a87bd', '#3885bc', '#3783bb', '#3681ba', '#3480b9', '#337eb8', '#327cb7', '#307ab6', '#2f79b5', '#2e77b5', '#2c75b4', '#2b73b3', '#2a71b2', '#2870b1', '#276eb0', '#266caf', '#246aae', '#2369ad', '#2267ac', '#2065ab', '#1f63a8', '#1e61a5', '#1d5fa2', '#1c5c9f', '#1b5a9c', '#1a5899', '#195696', '#185493', '#175290', '#15508d', '#144e8a', '#134c87', '#124984', '#114781', '#10457e', '#0f437b', '#0e4179', '#0d3f76', '#0c3d73', '#0a3b70', '#09386d', '#08366a', '#073467', '#063264', '#053061'], 'min': 0.0, 'max': 50.0}

# Add SSC layer to the map
m.addLayer(dataset.select('ssc').clip(lake), vis_params, 'SSC')

In [ ]:
# Select thermal infrared bands
tir1 = dataset.select('B10')  # 10.8-11.3 µm
tir2 = dataset.select('B11')  # 11.5-12.5 µm

# Calculate Kelvin temperature
kelvin = tir2.divide(10).add(273.15)

# Calculate Land Surface Temperature (LST)
lst = kelvin.divide(ee.Image(1).toFloat().divide(tir2.divide(14380).add(1).log()))

# Define emissivity and atmospheric correction
emissivity = ee.Image(0.98)
atm_corr = lst.multiply(0.99).add(0.11).multiply(emissivity).subtract(2.5)

# Calculate Water Surface Temperature (WST)
wst = atm_corr.subtract(273.15).rename('wst')

# Define visualization parameters for WST
wst_visualization = {
    'min': 0,
    'max': 30,
    'palette': [
        '0000FF',  # Blue
        '00FFFF',  # Cyan
        '00FF00',  # Green
        'FFFF00',  # Yellow
        'FFA500',  # Orange
        'FF0000'   # Red
    ]
}

# Add WST layer to the map
m.addLayer(wst.clip(lake), wst_visualization, 'Water Surface Temperature')

In [36]:
df = pd.read_csv(out_ndti_stats)
df

,mean,Lake_name,system:index
0,0.204702,Victoria,0
